In [1]:
# conda install -c rapidsai -c nvidia -c conda-forge dask-cuda cudatoolkit=11.6
# nvidia-smi
from sklearn.model_selection import train_test_split, KFold
import dask.dataframe as dd
import dask.multiprocessing
import dask
import numpy as np
from dask import delayed
import dask_image.imread
import dask_image.ndfilters
import dask_image.ndmeasure
import cv2
from glob import glob
import json
import dask.array as da
import time
import xgboost as xgb
import pandas as pd
import os
from dask import compute
from sklearn.metrics import f1_score
# f1_score(y_true, y_pred, average=[‘micro’, ‘macro’, ‘samples’,’weighted’ 중 하나 선택])
import tensorflow as tf

@delayed
def padding(data, array_len, col_len):
    pad = np.zeros((array_len, col_len))
    length = min(array_len, len(data))
    
    pad[:length] = data[:length]
    return pad

@delayed
def img_resize(img):
    img = cv2.resize(img, dsize=(224, 224), interpolation=cv2.INTER_AREA)
    img = img.astype(np.float32)/255
    return img

path = os.getcwd()
csv_features = ['내부 온도 1 평균', '내부 온도 1 최고', '내부 온도 1 최저', '내부 습도 1 평균', '내부 습도 1 최고', 
                '내부 습도 1 최저', '내부 이슬점 평균', '내부 이슬점 최고', '내부 이슬점 최저']

# from dask.distributed import Client, LocalCluster
# # client.close()
# cluster = LocalCluster()
# client = Client(cluster)
# print(cluster,client)
# client.close()
# http://localhost:8787/status

# from dask_cuda import LocalCUDACluster
# from dask.distributed import Client
# client.close()
# # http://127.0.0.1:34497/status
# cluster = LocalCUDACluster()
# client = Client(cluster)
# print(cluster,client)

/home/gob/miniconda3/envs/pro1/lib/python3.7/site-packages/pims/image_reader.py:26: RuntimeWarning: PIMS image_reader.py could not find scikit-image. Falling back to matplotlib's imread(), which uses floats instead of integers. This may break your scripts. 
(To ignore this warning, include the line "warnings.simplefilter("ignore", RuntimeWarning)" in your script.)
  warnings.warn(RuntimeWarning(ski_preferred))
2022-01-27 16:16:03.047947: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /home/gob/miniconda3/envs/pro1/lib/python3.7/site-packages/cv2/../../lib64:
2022-01-27 16:16:03.048018: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


In [2]:
from tensorflow.python.client import device_lib
device_lib.list_local_devices()

2022-01-27 16:16:04.332928: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-01-27 16:16:04.362204: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:925] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2022-01-27 16:16:04.362405: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /home/gob/miniconda3/envs/pro1/lib/python3.7/site-packages/cv2/../../lib64:
2022-01-27 16:16:04.362464: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dy

[name: "/device:CPU:0"
 device_type: "CPU"
 memory_limit: 268435456
 locality {
 }
 incarnation: 9929296599604544745
 xla_global_id: -1]

In [3]:
#making_label_set

crop = {'1':'딸기','2':'토마토','3':'파프리카','4':'오이','5':'고추','6':'시설포도'}
disease = {'1':{'a1':'딸기잿빛곰팡이병','a2':'딸기흰가루병','b1':'냉해피해','b6':'다량원소결핍 (N)','b7':'다량원소결핍 (P)','b8':'다량원소결핍 (K)'},
           '2':{'a5':'토마토흰가루병','a6':'토마토잿빛곰팡이병','b2':'열과','b3':'칼슘결핍','b6':'다량원소결핍 (N)','b7':'다량원소결핍 (P)','b8':'다량원소결핍 (K)'},
           '3':{'a9':'파프리카흰가루병','a10':'파프리카잘록병','b3':'칼슘결핍','b6':'다량원소결핍 (N)','b7':'다량원소결핍 (P)','b8':'다량원소결핍 (K)'},
           '4':{'a3':'오이노균병','a4':'오이흰가루병','b1':'냉해피해','b6':'다량원소결핍 (N)','b7':'다량원소결핍 (P)','b8':'다량원소결핍 (K)'},
           '5':{'a7':'고추탄저병','a8':'고추흰가루병','b3':'칼슘결핍','b6':'다량원소결핍 (N)','b7':'다량원소결핍 (P)','b8':'다량원소결핍 (K)'},
           '6':{'a11':'시설포도탄저병','a12':'시설포도노균병','b4':'일소피해','b5':'축과병'}}
risk = {'0':'정상','1':'초기','2':'중기','3':'말기'}


#ensemble_labler
label_description = {}
for key, value in disease.items():
    label_description[f'{key}_00_0'] = f'{crop[key]}_정상'
    for disease_code in value:
        for risk_code in risk:
            label = f'{key}_{disease_code}_{risk_code}'
            label_description[label] = f'{crop[key]}_{disease[key][disease_code]}_{risk[risk_code]}'

global ensemble_label_encoder
ensemble_label_encoder = {key:idx for idx, key in enumerate(label_description)}
ensemble_label_decoder = {val:key for key, val in ensemble_label_encoder.items()}

#crop_labler
crop_label_description = {}
for key, value in disease.items():
    crop_label_description[f'{key}'] = f'{crop[key]}'
    
global crop_label_encoder
crop_label_encoder = {key:idx for idx, key in enumerate(crop_label_description)}
crop_label_decoder = {val:key for key, val in crop_label_encoder.items()}


#disease_labler
disease_label_description = {}
for key, value in disease.items():
    disease_label_description[f'00'] = "정상"
    for disease_code,value in value.items():
        label = f'{disease_code}'
        disease_label_description[label] = f'{value}'

global disease_label_encoder
disease_label_encoder = {key:idx for idx, key in enumerate(disease_label_description)}
disease_label_decoder = {val:key for key, val in disease_label_encoder.items()}


#risk_labler
risk_label_description = {}
for key, value in risk.items():
    label = f'{key}'
    risk_label_description[label] = f'{value}'

global risk_label_encoder
risk_label_encoder = {key:idx for idx, key in enumerate(risk_label_description)}
risk_label_decoder = {val:key for key, val in risk_label_encoder.items()}





In [4]:
crop_label_description

{'1': '딸기', '2': '토마토', '3': '파프리카', '4': '오이', '5': '고추', '6': '시설포도'}

In [5]:
risk_label_description

{'0': '정상', '1': '초기', '2': '중기', '3': '말기'}

In [6]:
disease_label_description

{'00': '정상',
 'a1': '딸기잿빛곰팡이병',
 'a2': '딸기흰가루병',
 'b1': '냉해피해',
 'b6': '다량원소결핍 (N)',
 'b7': '다량원소결핍 (P)',
 'b8': '다량원소결핍 (K)',
 'a5': '토마토흰가루병',
 'a6': '토마토잿빛곰팡이병',
 'b2': '열과',
 'b3': '칼슘결핍',
 'a9': '파프리카흰가루병',
 'a10': '파프리카잘록병',
 'a3': '오이노균병',
 'a4': '오이흰가루병',
 'a7': '고추탄저병',
 'a8': '고추흰가루병',
 'a11': '시설포도탄저병',
 'a12': '시설포도노균병',
 'b4': '일소피해',
 'b5': '축과병'}

In [7]:
@delayed
def label_encoding(label):
    global ensemble_label_encoder
    encoded_label = ensemble_label_encoder[label]
    return encoded_label

@delayed
def label_encoding_crop(label):
    global crop_label_encoder
    encoded_label = crop_label_encoder[label]
    return encoded_label

@delayed
def label_encoding_disease(label):
    global disease_label_encoder
    encoded_label = disease_label_encoder[label]
    return encoded_label

@delayed
def label_encoding_risk(label):
    global risk_label_encoder
    encoded_label = risk_label_encoder[label]
    return encoded_label

def getlable(jsonpath,type):
    if type == "ensemble":
        with open(jsonpath, 'r') as f:
            json_file = json.load(f)

        crop = json_file['annotations']['crop']
        disease = json_file['annotations']['disease']
        risk = json_file['annotations']['risk']
        label = f'{crop}_{disease}_{risk}'
        return label

    elif type == "crop":
        with open(jsonpath, 'r') as f:
            json_file = json.load(f)
        crop = json_file['annotations']['crop']
        label = f'{crop}'
        return label

    elif type == "dc":
        with open(jsonpath, 'r') as f:
            json_file = json.load(f)
        disease = json_file['annotations']['disease']
        label = f'{disease}'
        return label

    elif type == "risk":  
        with open(jsonpath, 'r') as f:
            json_file = json.load(f)
        risk = json_file['annotations']['risk']
        label = f'{risk}'
        return label

    
def get_label_list(labelpath_list):
    labelarr = np.array([])
    labelarr_crop = np.array([])
    labelarr_dc = np.array([])
    labelarr_risk = np.array([])


    # labelarr = da.array([])
    for ind,json_path in enumerate(labelpath_list):
        # label = label_encoding(getlable(json_path))
        # label = da.array(np.array(label_encoding(getlable(json_path))))
        label = np.array(label_encoding(getlable(json_path,"ensemble")))
        labelarr = np.append(labelarr,label)

        label_crop = np.array(label_encoding_crop(getlable(json_path,"crop")))
        labelarr_crop = np.append(labelarr_crop,label_crop)

        label_dc = np.array(label_encoding_disease(getlable(json_path,"dc")))
        labelarr_dc = np.append(labelarr_dc,label_dc)

        label_risk = np.array(label_encoding_risk(getlable(json_path,"risk")))
        labelarr_risk = np.append(labelarr_risk,label_risk)


    return labelarr, labelarr_crop, labelarr_dc, labelarr_risk

In [8]:
@delayed
def get_all_image(imgpath_list,image_size):
    imgarr = np.empty((0,image_size,image_size,3), float)
    for ind,img_path in enumerate(imgpath_list):
        imgarr = delayed(np.append)(imgarr,img, axis = 0)
    return imgarr

In [9]:
%%time
@delayed
def imageresize(img):
    # img = dask.delayed(cv2.resize)(img, dsize=(224, 224), interpolation=cv2.INTER_AREA)
    # img = dask.delayed(img.astype(np.float32)/255)  ##픽셀값을 0~1사이로 정규화
    # # img = np.transpose(img, (2,0,1))
    # return dask.delayed(img.reshape)(-1,224,224,3)
    img = cv2.resize(img, dsize=(224, 224), interpolation=cv2.INTER_AREA)
    img = img.astype(np.float32)/255  ##픽셀값을 0~1사이로 정규화
    # img = np.transpose(img, (2,0,1))
    return np.array(img)
#making_img_set
row_img = dask_image.imread.imread(os.path.join(path,"data","train","*","*.jpg"))
train = [imageresize(img) for img in row_img]
train_x = np.array(dask.compute(*train))

CPU times: user 39.7 s, sys: 17.2 s, total: 56.9 s
Wall time: 22 s


In [10]:
%%time
label_list = sorted(glob(os.path.join(path,"data","train","*","*.json")))
y_train,y_train_crop,y_train_dc,y_train_risk, = get_label_list(label_list)
# y_train = y_train.rechunk(5767)
results = dask.compute(*y_train)
label = np.array(results)
# print(label.shape)
results_crop = dask.compute(*y_train_crop)
label_crop = np.array(results_crop)
#
results_dc = dask.compute(*y_train_dc)
label_disease = np.array(results_dc)
#
results_risk = dask.compute(*y_train_risk)
label_risk = np.array(results_risk)
label.shape,label_crop.shape,label_disease.shape,label_risk.shape

CPU times: user 3.62 s, sys: 375 ms, total: 4 s
Wall time: 3.89 s


((5767,), (5767,), (5767,), (5767,))

In [11]:
base_model = tf.keras.applications.MobileNetV2(input_shape=(224,224,3),
                                               include_top=False,
                                               weights='imagenet')
base_model.trainable = False
global_average_layer = tf.keras.layers.GlobalAveragePooling2D()
model = tf.keras.Sequential([
  base_model,
  global_average_layer,
  
])
model.compile()

2022-01-27 16:16:33.224083: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:925] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2022-01-27 16:16:33.224152: W tensorflow/core/common_runtime/gpu/gpu_device.cc:1850] Cannot dlopen some GPU libraries. Please make sure the missing libraries mentioned above are installed properly if you would like to use GPU. Follow the guide at https://www.tensorflow.org/install/gpu for how to download and setup the required libraries for your platform.
Skipping registering GPU devices...


In [12]:
%%time
batch = 100
data_length = len(train_x)
loop_num = int(np.floor(data_length/batch))

train_x_feature = dask.array.zeros((data_length,model(train_x[:1]).numpy().shape[1]))


count = 0
for i in range(loop_num):
    print(i)
    if i != loop_num-1:
        start_p = count*batch
        end_p = (count+1)*batch
        train_x_feature[start_p:end_p] = model(train_x[start_p:end_p]).numpy()
        
    else:
        start_p = count*batch
        end_p = (count+1)*batch
        train_x_feature[start_p:] = model(train_x[start_p:]).numpy()
            
    count += 1



0


2022-01-27 16:16:34.288718: W tensorflow/core/framework/cpu_allocator_impl.cc:82] Allocation of 60211200 exceeds 10% of free system memory.
2022-01-27 16:16:34.302666: W tensorflow/core/framework/cpu_allocator_impl.cc:82] Allocation of 160563200 exceeds 10% of free system memory.
2022-01-27 16:16:34.340160: W tensorflow/core/framework/cpu_allocator_impl.cc:82] Allocation of 160563200 exceeds 10% of free system memory.
2022-01-27 16:16:34.417330: W tensorflow/core/framework/cpu_allocator_impl.cc:82] Allocation of 160563200 exceeds 10% of free system memory.
2022-01-27 16:16:34.460064: W tensorflow/core/framework/cpu_allocator_impl.cc:82] Allocation of 160563200 exceeds 10% of free system memory.


1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
CPU times: user 15min, sys: 8min 46s, total: 23min 47s
Wall time: 1min 44s


## crop -> ensemble 모델

crop 기반 xgboost 학습

In [40]:
# 파라미터 조정
y_all = label_crop
y_all_final = label
cnn_out_dim = 1280
description = crop_label_description
decoder = crop_label_decoder
nclass = len(description)

In [41]:
X, X_test, y, y_test,y_ensemble,y_ensemble_test = train_test_split(train_x_feature, y_all,y_all_final, test_size=0.2, shuffle= True,stratify=y_all)
X = da.array(X).rechunk((1000,cnn_out_dim))
X_test = da.array(X_test).rechunk((1000,cnn_out_dim))
y = da.array(y).rechunk((1000,1))
y_test = da.array(y_test).rechunk((1000,1))
y_ensemble = da.array(y_ensemble).rechunk((1000,1))
y_ensemble_test = da.array(y_ensemble_test).rechunk((1000,1))

In [42]:
%%time
from dask.distributed import Client, LocalCluster
if __name__ == "__main__":
    with LocalCluster(dashboard_address=':3258') as cluster:
        with Client(cluster) as client:
            print(client)
            # X and y must be Dask dataframes or arrays
            

            dtrain = xgb.dask.DaskDMatrix(client, X, y,)

            params = {"verbosity": 0, 
            "tree_method": "hist", 
            "objective": 'multi:softmax',
            "num_class":nclass,
            }
            
            output = xgb.dask.train(
                client,
                params,
                dtrain,
                num_boost_round=10,
                evals=[(dtrain, "train")],
            )
            X_2_pred = xgb.dask.predict(client, output, X).compute()
            X_2_test_pred = xgb.dask.predict(client, output, X_test).compute()

<Client: 'tcp://127.0.0.1:36123' processes=4 threads=16, memory=24.59 GiB>


[16:45:48] task [xgboost.dask]:tcp://127.0.0.1:40015 got new rank 0
[16:45:48] task [xgboost.dask]:tcp://127.0.0.1:33325 got new rank 1


[0]	train-mlogloss:1.15786
[1]	train-mlogloss:0.83798
[2]	train-mlogloss:0.62804
[3]	train-mlogloss:0.48152
[4]	train-mlogloss:0.37242
[5]	train-mlogloss:0.29065
[6]	train-mlogloss:0.22948
[7]	train-mlogloss:0.18200
[8]	train-mlogloss:0.14550
[9]	train-mlogloss:0.11767
CPU times: user 3.93 s, sys: 1.64 s, total: 5.58 s
Wall time: 33.1 s


In [43]:
# 파라미터 조정
cnn_out_dim = 1281
description = label_description
decoder = ensemble_label_decoder
nclass = len(description)

In [44]:
X_2 = da.array(X_2_pred.reshape(-1,1)).rechunk((1000,1))
X_2_test = da.array(X_2_test_pred.reshape(-1,1)).rechunk((1000,1))

X_con = da.concatenate((X,X_2), axis=1).rechunk((1000,1281))
X_con_test = da.concatenate((X_test,X_2_test), axis=1).rechunk((1000,1281))

In [45]:
X_con_test

dask.array<rechunk-merge, shape=(1154, 1281), dtype=float64, chunksize=(1000, 1281), chunktype=numpy.ndarray>

In [51]:
%%time
from dask.distributed import Client, LocalCluster
if __name__ == "__main__":
    with LocalCluster(dashboard_address=':3258') as cluster:
        with Client(cluster) as client:
            print(client)
            # X and y must be Dask dataframes or arrays
            

            dtrain = xgb.dask.DaskDMatrix(client, X_con, y_ensemble,)

            params = {"verbosity": 0, 
            "tree_method": "hist", 
            "objective": 'multi:softmax',
            "num_class":nclass,
            }
            
            output = xgb.dask.train(
                client,
                params,
                dtrain,
                num_boost_round=10,
                evals=[(dtrain, "train")],
            )
            y_pred = xgb.dask.predict(client, output, X_con_test).compute()

<Client: 'tcp://127.0.0.1:46139' processes=4 threads=16, memory=24.59 GiB>


[16:50:18] task [xgboost.dask]:tcp://127.0.0.1:39755 got new rank 0
[16:50:18] task [xgboost.dask]:tcp://127.0.0.1:34991 got new rank 1


[0]	train-mlogloss:1.39222
[1]	train-mlogloss:0.78910
[2]	train-mlogloss:0.51872
[3]	train-mlogloss:0.37511
[4]	train-mlogloss:0.27842
[5]	train-mlogloss:0.21385
[6]	train-mlogloss:0.16762
[7]	train-mlogloss:0.13427
[8]	train-mlogloss:0.10871
[9]	train-mlogloss:0.08924
CPU times: user 11.5 s, sys: 380 ms, total: 11.9 s
Wall time: 1min 38s


In [52]:
from sklearn import metrics

# Print the confusion matrix
print(metrics.confusion_matrix(y_ensemble_test, y_pred))

# Print the precision and recall, among other metrics
print(metrics.classification_report(y_ensemble_test, y_pred, digits=3))

[[158   0   0   1   0   0   0   0   0   0   0   0   0   0   0   0   0   3
    0   0   0   0   0   0   0]
 [  2  23   1   1   0   0   0   1   0   0   0   1   0   0   0   0   1   1
    0   0   0   0   0   1   0]
 [  1   2  28   2   0   0   0   0   0   0   0   0   0   0   0   0   0   0
    0   0   0   1   0   0   0]
 [  0   0   1 204   6   2   0   2   5   1   4   1   0   1   0   2   0   1
    0   0   0   2   0   0   0]
 [  0   0   2  19   7   6   1   0   0   6   2   0   0   0   0   0   0   0
    0   0   0   0   0   0   0]
 [  0   0   0   5   9   2   0   0   1   0   0   1   0   0   0   2   0   0
    0   0   0   0   0   0   0]
 [  0   0   0   2   3   0   2   0   0   1   0   0   0   0   0   0   0   0
    0   0   0   0   0   0   0]
 [  0   0   0   7   2   2   0  10   1   3   3   0   1   0   0   0   0   0
    0   0   0   0   0   0   0]
 [  0   0   0  11   2   0   0   2   7   1   3   0   0   0   0   0   2   0
    0   0   0   0   0   0   0]
 [  0   0   0   3   0   0   0   1   1  12   3   2   0  

/home/gob/miniconda3/envs/pro1/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/gob/miniconda3/envs/pro1/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/gob/miniconda3/envs/pro1/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [53]:
answer = np.array([description[decoder[int(val)]] for val in y_ensemble_test])
predss = np.array([description[decoder[int(val)]] for val in y_pred])

new_crosstab = pd.crosstab(answer, predss, rownames=['answer'], colnames=['preds'])
new_crosstab

preds,고추_고추탄저병_중기,고추_다량원소결핍 (K)_초기,고추_다량원소결핍 (N)_초기,고추_다량원소결핍 (P)_초기,고추_정상,딸기_정상,시설포도_시설포도노균병_중기,시설포도_시설포도노균병_초기,시설포도_시설포도탄저병_초기,시설포도_일소피해_말기,...,토마토_정상,토마토_토마토흰가루병_중기,파프리카_다량원소결핍 (K)_초기,파프리카_다량원소결핍 (N)_초기,파프리카_다량원소결핍 (P)_초기,파프리카_정상,파프리카_칼슘결핍_초기,파프리카_파프리카흰가루병_말기,파프리카_파프리카흰가루병_중기,파프리카_파프리카흰가루병_초기
answer,,,,,,,,,,,,,,,,,,,,,
고추_고추탄저병_중기,12,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,2,0,0,0,0
고추_다량원소결핍 (K)_초기,0,13,2,0,1,0,0,0,0,0,...,0,0,0,0,0,13,2,0,0,2
고추_다량원소결핍 (N)_초기,0,3,9,0,0,0,0,0,0,0,...,0,0,0,2,0,5,3,0,2,2
고추_다량원소결핍 (P)_초기,0,0,0,38,0,0,0,0,0,0,...,0,0,0,0,0,3,0,0,0,0
고추_정상,0,0,1,0,8,0,0,0,0,0,...,0,0,0,0,0,3,0,0,0,0
딸기_정상,0,0,0,0,0,158,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0
시설포도_시설포도노균병_중기,0,0,0,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
시설포도_시설포도노균병_초기,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
시설포도_시설포도탄저병_중기,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [55]:
from sklearn.metrics import f1_score
f1_score(y_ensemble_test, y_pred,average='macro')

0.4816236310997495

## 전체모델 과적합 확인

In [56]:
# 파라미터 조정
y_all = label_disease
cnn_out_dim = 1280
description = label_description
decoder = ensemble_label_decoder
nclass = len(description)

In [62]:
X, X_test, y, y_test = train_test_split(train_x_feature, y_all, test_size=0.2, shuffle= True,stratify=y_all)
X = da.array(X).rechunk((1000,cnn_out_dim))
X_test = da.array(X_test).rechunk((1000,cnn_out_dim))
y = da.array(y).rechunk((1000,1))
y_test = da.array(y_test).rechunk((1000,1))

In [63]:
%%time
from dask.distributed import Client, LocalCluster
if __name__ == "__main__":
    with LocalCluster(dashboard_address=':3258') as cluster:
        with Client(cluster) as client:
            print(client)
            # X and y must be Dask dataframes or arrays
            

            dtrain = xgb.dask.DaskDMatrix(client, X, y,)

            params = {"verbosity": 0, 
            "tree_method": "hist", 
            "objective": 'multi:softmax',
            "num_class":nclass,
            }
            
            output = xgb.dask.train(
                client,
                params,
                dtrain,
                num_boost_round=10,
                evals=[(dtrain, "train")],
            )
            y_pred = xgb.dask.predict(client, output, dtrain).compute()
            y_pred_test = xgb.dask.predict(client, output, X_test).compute()

<Client: 'tcp://127.0.0.1:34609' processes=4 threads=16, memory=24.59 GiB>


[17:02:31] task [xgboost.dask]:tcp://127.0.0.1:41539 got new rank 0
[17:02:32] task [xgboost.dask]:tcp://127.0.0.1:33051 got new rank 1


[0]	train-mlogloss:1.80292
[1]	train-mlogloss:1.16482
[2]	train-mlogloss:0.72355
[3]	train-mlogloss:0.54134
[4]	train-mlogloss:0.41694
[5]	train-mlogloss:0.32583
[6]	train-mlogloss:0.25834
[7]	train-mlogloss:0.20667
[8]	train-mlogloss:0.16454
[9]	train-mlogloss:0.13145
CPU times: user 9.01 s, sys: 471 ms, total: 9.48 s
Wall time: 1min 22s


In [64]:
from sklearn import metrics

# Print the confusion matrix
print(metrics.confusion_matrix(y, y_pred))

# Print the precision and recall, among other metrics
print(metrics.classification_report(y, y_pred, digits=3))

[[3121    4   13    2    2    2    8    1    0    2    0    0]
 [  14  213    2    2    0    0    1    0    0    0    0    0]
 [  14    0  237    1    0    0    0    0    0    0    0    0]
 [   7    0    2  235    0    1    3    0    0    0    0    0]
 [   3    0    0    0  148    0    0    0    0    0    0    0]
 [   6    0    1    1    0  125    0    0    0    0    0    0]
 [  12    0    1    2    0    0  230    0    0    0    0    0]
 [   3    0    0    0    0    0    0   76    0    0    0    0]
 [   4    0    0    0    0    0    0    0   37    0    0    0]
 [   2    0    0    0    0    0    0    0    0   32    0    0]
 [   3    0    0    0    0    0    0    0    0    0   23    0]
 [   2    0    0    0    0    0    0    0    0    0    0   15]]
              precision    recall  f1-score   support

           0      0.978     0.989     0.984      3155
           4      0.982     0.918     0.949       232
           5      0.926     0.940     0.933       252
           6      0.967   

In [65]:
from sklearn import metrics

# Print the confusion matrix
print(metrics.confusion_matrix(y_test, y_pred_test))

# Print the precision and recall, among other metrics
print(metrics.classification_report(y_test, y_pred_test, digits=3))

[[732   8  12   8   0   6  17   2   2   1   1   0]
 [ 19  20   4  11   0   1   3   0   0   0   0   0]
 [ 21   2  30   3   0   4   2   0   1   0   0   0]
 [ 12  17   5  20   1   6   1   0   0   0   0   0]
 [  7   0   2   0  29   0   0   0   0   0   0   0]
 [  8   3   4   2   0  12   4   0   0   0   0   0]
 [ 14   5   6   3   0   1  32   0   0   1   0   0]
 [  2   0   0   0   0   0   0  16   1   0   1   0]
 [  4   0   0   0   0   0   0   0   7   0   0   0]
 [  8   0   0   0   0   0   0   0   0   0   0   0]
 [  6   0   0   0   0   0   0   0   0   0   0   0]
 [  3   0   0   0   0   0   0   0   0   0   0   1]]
              precision    recall  f1-score   support

           0      0.876     0.928     0.901       789
           4      0.364     0.345     0.354        58
           5      0.476     0.476     0.476        63
           6      0.426     0.323     0.367        62
           7      0.967     0.763     0.853        38
          10      0.400     0.364     0.381        33
        

In [66]:
from sklearn.metrics import f1_score
f1_score(y, y_pred,average='macro')


0.9540848603489009

In [67]:
from sklearn.metrics import f1_score
f1_score(y_test, y_pred_test,average='macro')


0.47827970065349507

## 전체모델 과적합 규제항 확인

In [70]:
# 파라미터 조정
y_all = label_disease
cnn_out_dim = 1280
description = label_description
decoder = ensemble_label_decoder
nclass = len(description)

In [71]:
X, X_test, y, y_test = train_test_split(train_x_feature, y_all, test_size=0.2, shuffle= True,stratify=y_all)
X = da.array(X).rechunk((1000,cnn_out_dim))
X_test = da.array(X_test).rechunk((1000,cnn_out_dim))
y = da.array(y).rechunk((1000,1))
y_test = da.array(y_test).rechunk((1000,1))

In [97]:
%%time
from dask.distributed import Client, LocalCluster
if __name__ == "__main__":
    with LocalCluster(dashboard_address=':3258') as cluster:
        with Client(cluster) as client:
            print(client)
            # X and y must be Dask dataframes or arrays
            

            dtrain = xgb.dask.DaskDMatrix(client, X, y,)

            params = {"verbosity": 0, 
            "tree_method": "hist", 
            "objective": 'multi:softmax',
            "max_depth": 4,
            "min_child_weight": 1,
            "num_class":nclass,
            "gamma": 1,
            "sub_sample": 0.5,
            "colsample_bytree": 0.5,
            "lambda": 1,
            "scale_pos_weight": 0.5,
            }
            
            output = xgb.dask.train(
                client,
                params,
                dtrain,
                num_boost_round=20,
                evals=[(dtrain, "train")],
            )
            y_pred = xgb.dask.predict(client, output, dtrain).compute()
            y_pred_test = xgb.dask.predict(client, output, X_test).compute()

<Client: 'tcp://127.0.0.1:45047' processes=4 threads=16, memory=24.59 GiB>


[17:13:19] task [xgboost.dask]:tcp://127.0.0.1:42775 got new rank 0
[17:13:19] task [xgboost.dask]:tcp://127.0.0.1:38483 got new rank 1


[0]	train-mlogloss:2.06030
[1]	train-mlogloss:1.12305
[2]	train-mlogloss:0.84952
[3]	train-mlogloss:0.70103
[4]	train-mlogloss:0.58806
[5]	train-mlogloss:0.50120
[6]	train-mlogloss:0.43450
[7]	train-mlogloss:0.37955
[8]	train-mlogloss:0.33158
[9]	train-mlogloss:0.28943
[10]	train-mlogloss:0.24929
[11]	train-mlogloss:0.21716
[12]	train-mlogloss:0.19098
[13]	train-mlogloss:0.16805
[14]	train-mlogloss:0.15059
[15]	train-mlogloss:0.13136
[16]	train-mlogloss:0.11894
[17]	train-mlogloss:0.10752
[18]	train-mlogloss:0.09723
[19]	train-mlogloss:0.08897
CPU times: user 10.9 s, sys: 3.95 s, total: 14.9 s
Wall time: 2min 2s


In [98]:
from sklearn import metrics

# Print the confusion matrix
print(metrics.confusion_matrix(y, y_pred))

# Print the precision and recall, among other metrics
print(metrics.classification_report(y, y_pred, digits=3))

[[3136    6    1    1    1    2    7    0    0    0    1    0]
 [   9  221    0    2    0    0    0    0    0    0    0    0]
 [   3    0  248    0    1    0    0    0    0    0    0    0]
 [   9    1    1  235    1    1    0    0    0    0    0    0]
 [   2    0    0    0  149    0    0    0    0    0    0    0]
 [   4    0    0    0    0  129    0    0    0    0    0    0]
 [   6    0    0    0    0    0  239    0    0    0    0    0]
 [   1    0    0    0    0    0    0   78    0    0    0    0]
 [   0    0    0    0    0    0    0    0   41    0    0    0]
 [   1    0    0    0    0    0    0    0    0   33    0    0]
 [   0    0    0    0    0    0    0    0    0    0   26    0]
 [   0    0    0    0    0    0    0    0    0    0    0   17]]
              precision    recall  f1-score   support

           0      0.989     0.994     0.991      3155
           4      0.969     0.953     0.961       232
           5      0.992     0.984     0.988       252
           6      0.987   

In [99]:
from sklearn import metrics

# Print the confusion matrix
print(metrics.confusion_matrix(y_test, y_pred_test))

# Print the precision and recall, among other metrics
print(metrics.classification_report(y_test, y_pred_test, digits=3))

[[746   7  10   4   4   2  14   0   0   1   1   0]
 [  8  22   4  21   0   2   1   0   0   0   0   0]
 [ 20   2  31   5   0   1   4   0   0   0   0   0]
 [ 13   9   2  30   0   6   2   0   0   0   0   0]
 [ 10   0   1   0  27   0   0   0   0   0   0   0]
 [  9   1   3   3   0  16   1   0   0   0   0   0]
 [ 14   1   3   6   0   2  36   0   0   0   0   0]
 [  4   0   0   0   0   0   0  15   0   0   1   0]
 [  5   0   0   0   0   0   1   0   4   0   0   1]
 [  7   0   0   0   0   0   0   0   0   1   0   0]
 [  3   0   0   0   1   0   0   0   0   1   1   0]
 [  1   0   0   0   0   0   0   0   1   0   0   2]]
              precision    recall  f1-score   support

           0      0.888     0.946     0.916       789
           4      0.524     0.379     0.440        58
           5      0.574     0.492     0.530        63
           6      0.435     0.484     0.458        62
           7      0.844     0.711     0.771        38
          10      0.552     0.485     0.516        33
        

In [100]:
from sklearn.metrics import f1_score
f1_score(y, y_pred,average='macro')


0.9831586096785142

In [101]:
from sklearn.metrics import f1_score
f1_score(y_test, y_pred_test,average='macro')


0.546586656703477